# Цель
Исследовать поведение пользователей в приложении Quiz Freeze.

# Задачи
1. Проверить, зависит ли вероятность оплаты от выбранного пользователем уровня сложности.
2. Проверить, различается ли временной промежуток между регистрацией и оплатой у групп пользователей с разным уровнем сложности.

# Предварительные шаги

#### Импортируем библиотеку и исходные данные

In [3]:
import pandas as pd
events = pd.read_csv('7_4_Events.csv')
purchase = pd.read_csv('purchase.csv')


# На всякий случай работать будем с копиями исходных таблиц.
events_df = events.copy()
purchase_df = purchase.copy()

#### Ознакомимся с исходными данными

In [4]:
events_df.head()

,id,event_type,selected_level,start_time,tutorial_id,user_id
0,28903,registration,NaN,2016-05-11T23:40:55,NaN,12583
1,28904,registration,NaN,2016-05-11T23:49:58,NaN,12584
2,28905,registration,NaN,2016-05-12T00:53:07,NaN,12585
3,28906,tutorial_start,NaN,2016-05-12T01:32:20,17562.0,12585
4,28907,tutorial_finish,NaN,2016-05-12T01:34:53,17562.0,12585


In [5]:
events_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252334 entries, 0 to 252333
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   id              252334 non-null  int64  
 1   event_type      252334 non-null  object 
 2   selected_level  31086 non-null   object 
 3   start_time      252334 non-null  object 
 4   tutorial_id     125103 non-null  float64
 5   user_id         252334 non-null  int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 11.6+ MB


In [6]:
purchase_df.head()

,id,user_id,event_datetime,amount
0,15674,12584,2016-05-12T10:34:16,100
1,15675,12985,2016-05-13T08:25:56,50
2,15676,12828,2016-05-13T16:33:46,50
3,15677,12598,2016-05-14T01:09:37,150
4,15678,13037,2016-05-14T01:24:46,100


In [7]:
purchase_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5956 entries, 0 to 5955
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              5956 non-null   int64 
 1   user_id         5956 non-null   int64 
 2   event_datetime  5956 non-null   object
 3   amount          5956 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 186.2+ KB


#### Преобразуем даты в формат datetime

In [8]:
events_df['start_time'] = pd.to_datetime(events_df['start_time'], errors='coerce')
purchase_df['event_datetime'] = pd.to_datetime(purchase_df['event_datetime'])

#### Оставим записи только по пользователям, зарегистрировавшимся в 2018 году

In [9]:
# Сначала создаем список пользователей, зарегистрировавшихся в 2018 году.
cond_1 = events_df['event_type'] == 'registration'
cond_2 = events_df['start_time'] >= '2018-01-01'
cond_3 = events_df['start_time'] < '2019-01-01'
registered_in_2018 = events_df[cond_1 & cond_2 & cond_3]['user_id'].to_list()


# Сставляем в таблицах только нужные записи.
events_df = events_df[events_df['user_id'].isin(registered_in_2018)]
purchase_df = purchase_df[purchase_df['user_id'].isin(registered_in_2018)]

#### Ознакомимся с обновленными таблицами

In [10]:
events_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66959 entries, 51405 to 118364
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              66959 non-null  int64         
 1   event_type      66959 non-null  object        
 2   selected_level  8342 non-null   object        
 3   start_time      66959 non-null  datetime64[ns]
 4   tutorial_id     32954 non-null  float64       
 5   user_id         66959 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 3.6+ MB


In [11]:
# Проверим, какие события есть в столбце event_type.
events_df['event_type'].unique()

array(['registration', 'tutorial_start', 'tutorial_finish',
       'level_choice', 'pack_choice'], dtype=object)

In [12]:
purchase_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1600 entries, 1171 to 2778
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              1600 non-null   int64         
 1   user_id         1600 non-null   int64         
 2   event_datetime  1600 non-null   datetime64[ns]
 3   amount          1600 non-null   int64         
dtypes: datetime64[ns](1), int64(3)
memory usage: 62.5 KB


# Задача 1. Проверить, зависит ли вероятность оплаты от выбранного пользователем уровня сложности.

In [13]:
# Узнаем, сколько всего уникальных пользователей выбрали уровень сложности.
events_df[events_df['event_type'] == 'level_choice']['user_id'].nunique()

8342

In [14]:
# Узнаем, сколько пользователей выбрали различные уровни сложности.
events_df['selected_level'].value_counts()

medium    4645
easy      2448
hard      1249
Name: selected_level, dtype: int64

#### Разделим пользователей на группы по выбору уровня сложности для дальнейшего анализа.

In [15]:
# Определим пользователей, выбравших легкий уровень сложности,
# и сверим их количество с полученными выше данными.
users_easy_level = events_df[events_df['selected_level'] == 'easy']['user_id'].unique()
set_users_easy_level = set(users_easy_level)
print(
    '{} пользователей выбрали легкий уровень сложности'.format(len(
        set_users_easy_level
    ))
)

2448 пользователей выбрали легкий уровень сложности


In [16]:
# То же самое для пользователей, выбравших средний уровень сложности.
users_medium_level = events_df[events_df['selected_level'] == 'medium']['user_id'].unique()
set_users_medium_level = set(users_medium_level)
print(
    '{} пользователей выбрали средний уровень сложности'.format(len(
        set_users_medium_level
    ))
)

4645 пользователей выбрали средний уровень сложности


In [17]:
# Теперь для пользователей, выбравших средний уровень сложности.
users_hard_level = events_df[events_df['selected_level'] == 'hard']['user_id'].unique()
set_users_hard_level = set(users_hard_level)
print(
    '{} пользователей выбрали трудный уровень сложности'.format(len(
        set_users_hard_level
    ))
)

1249 пользователей выбрали трудный уровень сложности


In [18]:
# Убедимся, что данные сходятся.
len(set_users_easy_level) + len(set_users_medium_level) + len(
    set_users_hard_level
) == events_df[events_df['event_type'] == 'level_choice']['user_id'].nunique()

True

#### Узнаем, какой процент пользователей, выбравших легкий уровень сложности, совершили оплату.

In [19]:
purchase_easy_level = purchase_df[
    purchase_df['user_id'].isin(users_easy_level)
]

# Сколько пользователей, выбравших легкий уровень сложности, совершили оплату.
purchase_easy_level['user_id'].nunique()

189

In [20]:
# Теперь найдем процент совершивших оплату.
percent_purchase_easy = purchase_easy_level['user_id'].nunique() / len(
    (set_users_easy_level)
)
print(
    'Из выбравших легкий уровень оплату совершили {:.2%} пользователей'.format(
        percent_purchase_easy
    )
)


Из выбравших легкий уровень оплату совершили 7.72% пользователей


#### Узнаем, какой процент пользователей, выбравших средний уровень сложности, совершили оплату.

In [21]:
purchase_medium_level = purchase_df[
    purchase_df['user_id'].isin(users_medium_level)
]

# Сколько совершили оплату.
purchase_medium_level['user_id'].nunique()

969

In [22]:
percent_purchase_medium = purchase_medium_level['user_id'].nunique() / len(
    set_users_medium_level
)
print(
    'Из выбравших средний уровень оплату совершили {:.2%} пользователей'.format(
        percent_purchase_medium
    )
)

Из выбравших средний уровень оплату совершили 20.86% пользователей


#### Узнаем, какой процент пользователей, выбравших трудный уровень сложности, совершили оплату.

In [23]:
purchase_hard_level = purchase_df[
    purchase_df['user_id'].isin(users_hard_level)
]

# Сколько совершили оплату.
purchase_hard_level['user_id'].nunique()

442

In [24]:
percent_purchase_hard = purchase_hard_level['user_id'].nunique() / len(
    set_users_hard_level
)
print(
    'Из выбравших трудный уровень оплату совершили {:.2%} пользователей'.format(
        percent_purchase_hard
    )
)

Из выбравших трудный уровень оплату совершили 35.39% пользователей


#### Вывод: 

Из выбравших легкий уровень оплату совершили 7,72% пользователей, из выбравших средний уровень - 20,86%, из выбравших трудный уровень - 35,39%. 

Таким образом, чем выше выбранный пользователем уровень сложности, тем выше верояность покупки им платных вопросов.

# Задача 2. Проверить, различается ли временной промежуток между регистрацией и оплатой у групп пользователей с разным уровнем сложности.

Для каждой группы пользователей сделаем по два датафрейма: один — с событиями выбора уровня сложности, другой — с событиями оплаты. Затем объединим эти датафреймы в рамках каждой группы.

In [25]:
# Датафрейм с событиями оплаты мы ранее уже разбили на три датафрейма по группам пользователей.
# Это датафреймы purchase_easy_level, purchase_medium_level и purchase_hard_level.
# Аналогично разобьем датафрейм с событиями, а затем оставим в датафреймах только событие registration.
events_easy_level = events_df[events_df['user_id'].isin(users_easy_level)]
reg_easy_level = events_easy_level[events_easy_level['event_type'] == 'registration']

events_medium_level = events_df[events_df['user_id'].isin(users_medium_level)]
reg_medium_level = events_medium_level[events_medium_level['event_type'] == 'registration']

events_hard_level = events_df[events_df['user_id'].isin(users_hard_level)]
reg_hard_level = events_hard_level[events_hard_level['event_type'] == 'registration']

Перед тем, как объединять датафреймы, ознакомимся с их структурой на примере одной группы.

In [26]:
reg_easy_level.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2448 entries, 51430 to 118352
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              2448 non-null   int64         
 1   event_type      2448 non-null   object        
 2   selected_level  0 non-null      object        
 3   start_time      2448 non-null   datetime64[ns]
 4   tutorial_id     0 non-null      float64       
 5   user_id         2448 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 133.9+ KB


In [27]:
purchase_easy_level.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 189 entries, 1180 to 2767
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              189 non-null    int64         
 1   user_id         189 non-null    int64         
 2   event_datetime  189 non-null    datetime64[ns]
 3   amount          189 non-null    int64         
dtypes: datetime64[ns](1), int64(3)
memory usage: 7.4 KB


In [28]:
reg_easy_level.head()

,id,event_type,selected_level,start_time,tutorial_id,user_id
51430,80333,registration,NaN,2018-01-01 23:18:46,NaN,27842
51445,80348,registration,NaN,2018-01-02 07:03:55,NaN,27849
51469,80372,registration,NaN,2018-01-02 15:47:14,NaN,27853
51485,80388,registration,NaN,2018-01-02 21:30:27,NaN,27859
51501,80404,registration,NaN,2018-01-03 03:53:48,NaN,27863


In [29]:
purchase_easy_level.head()

,id,user_id,event_datetime,amount
1180,16854,27884,2018-01-08 19:37:34,150
1191,16865,28182,2018-01-12 02:46:01,200
1193,16867,28207,2018-01-12 21:00:24,150
1199,16873,28090,2018-01-15 23:42:55,100
1206,16880,28378,2018-01-18 02:11:41,100


#### Перейдем к соединению датафреймов. 

Начнем с пользователей, выбравших легкий уровень.

In [30]:
# Нас интересуют только пользователи, совершившие оплату, 
# поэтому выберем тип соединения inner.
reg_purchase_easy_df = reg_easy_level.merge(
    purchase_easy_level, on='user_id', how='inner'
)

# Удалим лишние столбцы и добавим столбец timedelta.
reg_purchase_easy_df = reg_purchase_easy_df[
    ['user_id', 'start_time', 'event_datetime']].rename(
        columns={'start_time': 'reg_time', 'event_datetime': 'purchase_time'}
)
reg_purchase_easy_df['timedelta'] = reg_purchase_easy_df['purchase_time'] - \
    reg_purchase_easy_df['reg_time']

# Выведем статистические показатели для временного интервала между
# регистрацией и оплатой.
reg_purchase_easy_df['timedelta'].describe()

count                          189
mean     3 days 22:10:23.211640211
std      2 days 07:14:41.062010764
min                0 days 04:36:58
25%                2 days 01:12:12
50%                3 days 11:00:23
75%                5 days 10:24:59
max               11 days 00:35:04
Name: timedelta, dtype: object

Теперь для пользователей, выбравших средний уровень.

In [31]:
reg_purchase_medium_df = reg_medium_level.merge(
    purchase_medium_level, on='user_id', how='inner'
)

# Удалим лишние столбцы и добавим столбец timedelta.
reg_purchase_medium_df = reg_purchase_medium_df[
    ['user_id', 'start_time', 'event_datetime']].rename(
        columns={'start_time': 'reg_time', 'event_datetime': 'purchase_time'}
)
reg_purchase_medium_df['timedelta'] = reg_purchase_medium_df['purchase_time'] - \
    reg_purchase_medium_df['reg_time']

# Выведем статистические показатели для временного интервала между
# регистрацией и оплатой.
reg_purchase_medium_df['timedelta'].describe()

count                          969
mean     4 days 06:12:06.576883384
std      2 days 06:25:57.480868026
min                0 days 08:39:24
25%                2 days 08:46:51
50%                4 days 03:35:26
75%                5 days 23:51:27
max               10 days 20:34:02
Name: timedelta, dtype: object

Наконец, для пользователей, выбравших трудный уровень.

In [32]:
reg_purchase_hard_df = reg_hard_level.merge(
    purchase_hard_level, on='user_id', how='inner'
)

# Удалим лишние столбцы и добавим столбец timedelta.
reg_purchase_hard_df = reg_purchase_hard_df[
    ['user_id', 'start_time', 'event_datetime']].rename(
        columns={'start_time': 'reg_time', 'event_datetime': 'purchase_time'}
    )

reg_purchase_hard_df['timedelta'] = reg_purchase_hard_df['purchase_time'] - \
    reg_purchase_hard_df['reg_time']

# Выведем статистические показатели для временного интервала между
# регистрацией и оплатой.    
reg_purchase_hard_df['timedelta'].describe()

count                          442
mean     3 days 14:55:19.257918552
std      1 days 22:22:52.441896774
min                0 days 09:41:39
25%         1 days 23:36:25.500000
50%         3 days 10:10:04.500000
75%         5 days 03:30:07.750000
max                8 days 14:21:29
Name: timedelta, dtype: object

#### Вывод: 

Выбравшим легкий уровень пользователям для перехода от регистрации к оплате в среднем требуется около 3 дней 22 часов, средний уровень - около 4 дней 6 часов, трудный уровень - около 3 дней 15 часов.

Таким образом, разница в средних показателях есть. Но с учетом стандартных отклонений в каждой группе (около 2 суток) ее нельзя назвать существенной. Кроме того, между выбором уровня сложности и временным интервалом между событиями registration и purchase не наблюдается прямой или обратной зависимости.

# Итог

1. Зависит ли вероятность оплаты от выбранного пользователем уровня сложности?

Да, чем выше выбранный пользователем уровень сложности, тем выше верояность покупки им платных вопросов.
Причем вероятность растет сильно: из выбравших легкий уровень оплату совершили 7,72% пользователей, из выбравших средний уровень - 20,86%, из выбравших трудный уровень - 35,39%. 

2. Существует ли разница во времени между событиями регистрации и оплаты для разных групп пользователей с разным уровнем сложности?

Разница незначительная, а также между выбором уровня сложности и временным интервалом между событиями registration и purchase не наблюдается прямой или обратной зависимости.

# Общий вывод

Наиболее выгодными для компании являются пользователи, выбирающие трудный уровень вопросов. 